# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="event_data/data_sample.png">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
#copied from Lesson 3 Exercise 1

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
#copied from Lesson 3 Exercise 1

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

As we creating tables based on queries, we shall first look at the queries to see what data is needed.



## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

`"SELECT artist, song, length FROM sessions WHERE sessionId = 338 AND itemInSession = 4"`

Question 1 states that we need the columns:
- artist, 
- song, 
- length
- sessionId
- itemInSession

Since our question is looking for `sessionId` and `itemInSession`, these columns compose our primary key. As we are looking for `sessionId`, this shall be our partition key. It is used to distribute data across the nodes in the Cassandra cluster.

`itemInSession` is our clustering column. The table can be called `sessions`




In [8]:
query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(artist text, \
                  song text, \
                  length float, \
                  sessionId int, \
                  itemInSession int, \
                  PRIMARY KEY (sessionid, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
#drop table statement in case table has to be redone for some reason
#query = "drop table sessions"
#try:
#    rows = session.execute(query)
#except Exception as e:
#    print(e)

For the `INSERT`statement, we have to know which column element should be assigned for each column in the INSERT statement. Therefore, let us import `pandas` and `read_csv`.

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv("event_datafile_new.csv")

In [12]:
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Hoobastank,Cierra,F,0,Finley,241.3971,free,"Richmond, VA",132,Say The Same,96
1,Mark Knopfler,Cierra,F,1,Finley,249.3122,free,"Richmond, VA",132,Why Aye Man,96


In [13]:
counter = 0

for column in df.columns:
    print(counter, column)
    counter += 1


0 artist
1 firstName
2 gender
3 itemInSession
4 lastName
5 length
6 level
7 location
8 sessionId
9 song
10 userId


In [14]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sessions (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, 
                        (line[0], #artist 
                         line[9], #song 
                         float(line[5]), #length
                         int(line[8]), #sessionId
                         int(line[3])  #itemInSession
                        ))

In [15]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "SELECT artist, song, length FROM sessions WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)


Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

`SELECT artist, song, firstName, lastName, itemInSession FROM users WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession`

Question 2 states that we need the columns:
- artist, 
- song, 
- firstName
- lastName
- userId
- sessionId
- itemInSession

`userId` and `sessionId` together form the partition key, facilitating data distribution across the cluster. The primary key is composite, consisting of both a partition key `userId` and clustering column `sessionId`.

`itemInSession` is a clustering column. This allows you to use `ORDER BY itemInSession` in the query.

The table can be called `users`


In [16]:
query = "CREATE TABLE IF NOT EXISTS users "
query = query + "(artist text, \
                  song text, \
                  firstName text, \
                  lastName text, \
                  userId int, \
                  sessionId int, \
                  itemInSession int, \
                  PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:

#drop table statement in case table has to be redone for some reason
#query = "drop table users"
#try:
#    rows = session.execute(query)
#except Exception as e:
#    print(e)


In [18]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO users (artist, song, firstName, lastName, userId, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, 
                        (line[0], #artist 
                         line[9], #song 
                         line[1], #firstName,
                         line[4], #lastName,
                         int(line[10]), #userId
                         int(line[8]), #sessionId
                         int(line[3])  #itemInSession
                        ))

In [19]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "SELECT artist, \
                song, \
                firstName, \
                lastName, \
                itemInSession \
                FROM users \
                WHERE \
                    userId = 10 AND sessionId = 182 \
                ORDER BY \
                    itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Question 3 states that we need the columns:
- firstName,
- lastName,
- song

We can create a composite primary key of `song` and `user_id` `(PRIMARY KEY (song, user_id))`. `song` column is the partition key and `user_id` is the clustering column. As I want to perform queries that filter or ordering by `song`, this is my partition key.

This table shall be calles `songs` .

`SELECT firstName, lastName, song FROM songs WHERE song = "All Hands Against His Own"`

In [20]:
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Hoobastank,Cierra,F,0,Finley,241.39710,free,"Richmond, VA",132,Say The Same,96
1,Mark Knopfler,Cierra,F,1,Finley,249.31220,free,"Richmond, VA",132,Why Aye Man,96
2,Mogwai,Cierra,F,2,Finley,341.28934,free,"Richmond, VA",132,We're No Here,96
3,The Casualties,Cierra,F,3,Finley,181.49832,free,"Richmond, VA",132,Punx Unite,96
4,The Living End,Ryan,M,0,Smith,188.62975,free,"San Jose-Sunnyvale-Santa Clara, CA",433,Roll On (Album Version),26


In [23]:
query = "CREATE TABLE IF NOT EXISTS songs "
query = query + "(user_id int, \
                  firstName text, \
                  lastName text, \
                  song text, \
                  PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [22]:
#drop table statement in case table has to be redone for some reason
#query = "drop table songs"
#try:
#    rows = session.execute(query)
#except Exception as e:
#    print(e)


In [24]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs (user_id, firstName, lastName, song)"
        query = query + " VALUES (%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, 
                        (int(line[10]), #userId
                         line[1], #firstName,
                         line[4], #lastName,
                         line[9] #song
                        ))

#### Do a SELECT to verify that the data have been inserted into each table

In [25]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table


query = "SELECT user_id, \
                firstName, \
                lastName, \
                song \
                FROM songs \
                WHERE \
                    song ='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(user_id=29, firstname='Jacqueline', lastname='Lynch', song='All Hands Against His Own')
Row(user_id=80, firstname='Tegan', lastname='Levine', song='All Hands Against His Own')
Row(user_id=95, firstname='Sara', lastname='Johnson', song='All Hands Against His Own')


### Drop the tables before closing out the sessions

In [26]:
## TO-DO: Drop the table before closing out the sessions

In [28]:
query = "drop table sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


query = "drop table users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


query = "drop table songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()